# SQL Overview
SQL (Structured Query Language) is a domain-specific language used in programming and designed for managing data held in a relational database management system, or for stream processing in a relational data stream management system. 

**Important Note:** I am using a MySQL database for all examples in this notebook. SQL queries differ slightly from database to database, I detail other db examples in the databases notebook inside of anaconda-projects/data-science/fundamentals

1. **[Basic SQL Statements](#s1)**
2. **[Creating & Editing Tables In SQL](#s2)**
3. **[Combining Tables](#s3)**
4. **[Functions](#s4)**
5. **[SubQueries](#s5)**
6. **[User Stored Funtions, Variables, Scripts, and Stored Procedures3. SubQueries](#s6)**

These are good refresher [Tutorials](https://www.youtube.com/watch?v=a9W7OpS4LfI&list=PLyuRouwmQCjlXvBkTfGeDTq79r9_GoMt9&ab_channel=SteveGriffith-Prof3ssorSt3v3) videos

In [11]:
import requests
import numpy as np
import pandas as pd
import mysql.connector 

conn = mysql.connector.connect(host='localhost',
                               user='root',
                               password='InfoDump21!')

db1 = 'netflix'
db2 = 'band'
db3 = 'classicmodels'
limit = 5

print(conn)

cur = conn.cursor()
cur.execute(f'CREATE DATABASE IF NOT EXISTS {db1};')

print(f'\nConnection to `{db1}` Database Created Successfully')

cur.close()
conn.close()
print(f'\n`{db1}` Database Connection Closed.\n')


Connection to `netflix` Database Created Successfully

`netflix` Database Connection Closed.



In [12]:
def run_query(query, db):
    try:
        conn = mysql.connector.connect(host='localhost',
                                       user='root',
                                       password='InfoDump21!',
                                       database=db)
        return pd.read_sql_query(query, conn)
    except:
        print(f'Error: could not connect to {db} database.')

def run_update(query, db, mode):
    conn = mysql.connector.connect(host='localhost',
                                   user='root',
                                   password='InfoDump21!',
                                   database=db)
    
    # Create cursor and use it to perform queries
    cur = conn.cursor()
    cur.execute(query)
    
    if (mode == 'insert') or (mode == 'delete') or (mode == 'update') or (mode == 'replace'):
        conn.commit()
    
    cur.close()
    conn.close()

<a id="s1"></a>
## 1. Basic SQL Statements
All SQL statements are run using SQL keyword which by convention are usually all in caps, however this is not necessary for them to work.

### USE
Selects the database you want to use, in the examples here I have already selected this inside of the run_query function

### DESCRIBE
Gives the field characteristics of a database tables

In [203]:
run_query('DESCRIBE movies', db1)

,Field,Type,Null,Key,Default,Extra
0,id,b'int',YES,,None,
1,title,b'varchar(255)',YES,,None,
2,director,b'varchar(100)',YES,,None,
3,release_date,b'int',YES,,None,
4,runtime,b'int',YES,,None,
5,rating,b'varchar(15)',YES,,None,
6,category,b'varchar(50)',YES,,None,


### SELECT
Used when wanting to select or pull information from a database table.

Select statements always have a **FROM** keyword as well to signify which table to pull from

### LIMIT
Used to limit the number of rows returned. A single value limits to only that number (i.e. LIMIT 10 limits to 10 rows returned) whereas (LIMIT 5, 10 starts at the 6th returned value and returns the next 10 rows)

In [204]:
run_query(f'SELECT * FROM movies LIMIT {limit}', db1)

,id,title,director,release_date,runtime,rating,category
0,81145628,Norm of the North: King Sized Adventure,Richard Finn,2019,90,TV-PG,Children & Family Movies
1,80125979,#realityhigh,Fernando Lebrija,2017,99,TV-14,Comedies
2,70304989,Automata,Gabe Ibanez,2014,110,R,International Movies
3,80164077,Fabrizio Copano: Solo pienso en mi,Rodrigo Toro,2017,60,TV-MA,Stand-Up Comedy
4,70304990,Good People,Henrik Ruben Genz,2014,90,R,Action & Adventure


### WHERE
Used to conditinally filter by fields (column values)

In [205]:
query = f"""
SELECT *
FROM movies
WHERE release_date = 2019
LIMIT {5}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,81145628,Norm of the North: King Sized Adventure,Richard Finn,2019,90,TV-PG,Children & Family Movies
1,81154455,Article 15,Anubhav Sinha,2019,125,TV-MA,Dramas
2,81132437,Kill Me If You Dare,Senol Sonmez,2019,100,TV-14,Comedies
3,81078908,The World We Make,Brian Baugh,2019,108,PG,Dramas
4,81155784,Watchman,A. L. Vijay,2019,93,TV-14,Comedies


### LIKE
Allows the use of wildcard symbols (think regex) to match specific parts of a string, this link has some of the more common exampels of LIKE's usage: [LIKE examples](https://www.tutorialspoint.com/sql/sql-like-clause.htm)

In [206]:
# This query finds all movies with the word 'star' in it
# note that case-sensitivity is determined on table creation

query = f"""
SELECT * 
FROM movies
WHERE title LIKE '%star%'
LIMIT {limit}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,70267838,20 Feet From Stardom,Morgan Neville,2013,91,PG-13,Documentaries
1,80092857,Pup Star,Robert Vince,2016,92,G,Children & Family Movies
2,80189954,Pup Star: Better 2Gether,Robert Vince,2017,93,PG,Children & Family Movies
3,80998890,Upstarts,Udai Singh Pawar,2019,112,TV-14,Comedies
4,80112370,Barbie Star Light Adventure,Andrew Tan,2016,79,TV-Y,Children & Family Movies


### AND/OR 
These are two conditional statements allowing for more required conditions within queries. AND means that all options must exist to be true, OR means either can exist to be true

In [207]:
# This query searches for movies with star in the title with length greater than 90 minutes
query = f"""
SELECT * 
FROM movies
WHERE title LIKE '%star%'
AND (runtime > 100 OR category LIKE '%Comedies%')
LIMIT {limit}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,80998890,Upstarts,Udai Singh Pawar,2019,112,TV-14,Comedies
1,28631236,Superstar,Bruce McCulloch,1999,82,PG-13,Comedies
2,81010865,Starting Over Again,Olivia M. Lamasan,2014,128,TV-14,Dramas
3,80245408,Secret Superstar,Advait Chandan,2017,150,TV-14,Dramas
4,70117293,The Men Who Stare at Goats,Grant Heslov,2009,94,R,Comedies


### ORDER BY
Used to sort the data by field names (numerically or alphabetically) in either ascending (ASC) or descending (DESC) orders

In [208]:
# This query searches for movies with star in the title with length greater than 90 minutes
query = f"""
SELECT * 
FROM movies
WHERE title LIKE '%star%'
AND 100
ORDER BY runtime DESC
LIMIT {limit}
"""
run_query(query, db1)

,id,title,director,release_date,runtime,rating,category
0,80192018,Star Wars: Episode VIII: The Last Jedi,Rian Johnson,2017,152,PG-13,Action & Adventure
1,80245408,Secret Superstar,Advait Chandan,2017,150,TV-14,Dramas
2,81046962,Solo: A Star Wars Story (Spanish Version),Ron Howard,2018,135,PG-13,Action & Adventure
3,80220814,Solo: A Star Wars Story,Ron Howard,2018,135,PG-13,Action & Adventure
4,81010865,Starting Over Again,Olivia M. Lamasan,2014,128,TV-14,Dramas


### Aliasing using AS
The AS command is used to rename a column or table with an alias which only exists for the duration of the query. This is used often when field names are confusion or really long. 

In [209]:
# In this example the field name 'director' or 'release_date' is changed to 'dir' and 'date'
# the movies table is also changed to 'm'
# note that table aliases don't work within their respective queries with WHERE etc.,
# below WHERE release_date (or m.release_date) works, but date or m.date would not

query = f"""
SELECT m.title, m.director AS dir, m.release_date AS date
FROM `movies` AS m
WHERE release_date = 2019
LIMIT {limit}
"""
run_query(query, db1)

,title,dir,date
0,Norm of the North: King Sized Adventure,Richard Finn,2019
1,Article 15,Anubhav Sinha,2019
2,Kill Me If You Dare,Senol Sonmez,2019
3,The World We Make,Brian Baugh,2019
4,Watchman,A. L. Vijay,2019


### DISTINCT 
Used to return the unique values in a column

In [210]:
# This query pulls all possible release dates for the movies 

query = f"""
SELECT DISTINCT release_date
FROM movies
ORDER BY release_date DESC
LIMIT {limit}
"""
run_query(query, db1)

,release_date
0,2020
1,2019
2,2018
3,2017
4,2016


### BETWEEN
Used to filtering a range of data

In [36]:
# The first example shows simple conditions using WHERE/AND

query1 = f"""
SELECT * 
FROM movies
WHERE release_date >= 1982 AND release_date <= 1994
ORDER BY release_date ASC
LIMIT {limit}
"""
run_query(query1, db1)
          
          
# Here BETWEEN is used to filter in a cleaner manner than above
query2 = f"""
SELECT * 
FROM movies
WHERE release_date BETWEEN 1982 AND 1994
ORDER BY release_date ASC
LIMIT {limit}
"""
run_query(query2, db1)

,id,title,director,release_date,runtime,rating,category
0,70124316,Five Elements Ninjas,Cheh Chang,1982,104,R,Action & Adventure
1,419733,The Dark Crystal,Frank Oz,1982,93,PG,Children & Family Movies
2,81168343,The lynx,Stanisaw Rozewicz,1982,82,TV-14,Dramas
3,70053010,Bazaar,Sagar Sarhadi,1982,121,TV-14,Dramas
4,80236778,Monty Python: Live at The Hollywood Bowl,Terry Hughes,1982,80,R,Comedies


### IN
Filter on values without explicitly setting equality, used when multple values are desired (using OR statement)

In [41]:
# The first example filters out specific movies by exact runtimes, but is verbose
query1 = f"""
SELECT * 
FROM movies
WHERE runtime = 100
OR runtime = 121
OR runtime = 141
LIMIT {limit}
"""
run_query(query1, db1)

# Using IN the above query can be done more efficiently
query1 = f"""
SELECT * 
FROM movies
WHERE runtime IN (100, 121, 141)
LIMIT {limit}
"""
run_query(query1, db1)

,id,title,director,release_date,runtime,rating,category
0,81132437,Kill Me If You Dare,Senol Sonmez,2019,100,TV-14,Comedies
1,80005444,Print the Legend,Luis Lopez,2014,100,TV-14,Documentaries
2,81080769,Do Paise Ki Dhoop Chaar Aane Ki Baarish,Deepti Naval,2009,121,TV-14,Dramas
3,80993149,Maynard,Samuel D. Pollard,2017,100,TV-PG,Documentaries
4,81016337,Gie,Riri Riza,2005,141,TV-14,Dramas


### IF & NULLIF
IF statements allow for column values to be set conditionally in an if/then pattern. Only one condition can be given and pass/fail results are allowed. The functionality of the IF statement is similar to IF in Excel

For multiple conditional statements in SQL, see the CASE statment below

In [4]:
# In the below example the if statement searches for 
# any movie title with 'the' in it and put it 
# in a column called 'THE MOVIE' if no match for 'the'
# put in NULL

query1 = f"""
SELECT 
m.id, 
m.title,
IF(m.title LIKE '%the%', 'THE MOVIE', NULL) as message
FROM movies as m 
LIMIT {limit}
"""
run_query(query1, db1)

,id,title,message
0,81145628,Norm of the North: King Sized Adventure,THE MOVIE
1,80125979,#realityhigh,None
2,70304989,Automata,None
3,80164077,Fabrizio Copano: Solo pienso en mi,None
4,70304990,Good People,None


In [5]:
# NULLIF returns null if the condition isn't met by default
query1 = f"""
SELECT 
m.id, 
m.title,
NULLIF(release_date, 2017) as 2017_nulled
FROM movies as m 
LIMIT {limit}
"""
run_query(query1, db1)

,id,title,2017_nulled
0,81145628,Norm of the North: King Sized Adventure,2019.0
1,80125979,#realityhigh,NaN
2,70304989,Automata,2014.0
3,80164077,Fabrizio Copano: Solo pienso en mi,NaN
4,70304990,Good People,2014.0


### CASE
The CASE statement provides multi-conditional filtering in SQL similar in function to an traditional 'if' statment in most programming languages.


* Case statements begin with CASE and end with END
* All conditions are use WHEN and THEN format and go in between CASE...END
* WHEN/THEN conditions can be stacked and can be finished with an ELSE statement
* Case statements can be surrounded by parenthesis for readibility if desired (not required)

In [6]:
query1 = f"""
SELECT 
id,
title,
category as genre,
runtime,
CASE
WHEN runtime < 60 THEN 'short'
WHEN runtime BETWEEN 60 AND 90 THEN 'average'
ELSE 'long'
END as length

FROM movies

"""
run_query(query1, db1)

,id,title,genre,runtime,length
0,81145628,Norm of the North: King Sized Adventure,Children & Family Movies,90,average
1,80125979,#realityhigh,Comedies,99,long
2,70304989,Automata,International Movies,110,long
3,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,60,average
4,70304990,Good People,Action & Adventure,90,average
...,...,...,...,...,...
4120,80093107,Toro,Dramas,106,long
4121,80085438,Frank and Cindy,Documentaries,70,average
4122,80085439,Frank and Cindy,Comedies,102,long
4123,80011846,Iverson,Documentaries,88,average


---
<a id="s2"></a>
## 2. Creating & Editing Tables In SQL
There are a number of different keywords used for creating, removing, and changing table content including:

**CREATE** - allows tables to be created from existing tables or from scratch

**DROP** - keyword used for deleting databases, tables, and columns from tables

**ALTER** - keyword used for altering table field (column) types and characterstics (for the table itself, not the values)

**INSERT** - inserts BRAND NEW records into a table (i.e. new row)

**UPDATE** - modifies exisiting table field values

**DELETE** - deletes existing table records (i.e. entire rows)

**REPLACE** - works exactly like INSERT, except that if an old row in the table has the same value as a new row for a PRIMARY KEY or a UNIQUE index, the old row is deleted before the new row is inserted. NOTE that this is not the same as the REPLACE() function which replaces all occurences of a substring in a string. 

### CREATE a table from an existing table

In [212]:
# Create a table called `comedies` to store all the comedies in the movies table

query = f"""
CREATE TABLE IF NOT EXISTS comedies AS
SELECT title, director, release_date AS date, runtime, rating FROM `movies` 
WHERE category = 'Comedies';
"""

test1 = f"""
SELECT COUNT(*) FROM comedies
LIMIT {limit};
"""

test2 = f"""
SELECT * FROM comedies
LIMIT {limit};
"""

run_update(query, db1, 'create')

print('Number of Rows Returned:')
print(run_query(test1, db1))
run_query(test2, db1)

Number of Rows Returned:
   COUNT(*)
0       787


,title,director,date,runtime,rating
0,#realityhigh,Fernando Lebrija,2017,99,TV-14
1,Manhattan Romance,Tom O'Brien,2014,98,TV-14
2,Sierra Burgess Is A Loser,Ian Samuels,2018,106,PG-13
3,Care of Kancharapalem,Maha Venkatesh,2018,142,TV-14
4,Ee Nagaraniki Emaindi,Tharun Bhascker,2018,133,TV-14


### CREATE  a table from scratch

In [47]:
# Here I create a table called memebers for a new database called band

create_table = f"""
CREATE TABLE IF NOT EXISTS members(
member_id INT NOT NULL AUTO_INCREMENT,
fname VARCHAR(50) NOT NULL,
lname VARCHAR(50) NOT NULL,
instrument VARCHAR(50) NOT NULL,
PRIMARY KEY (member_id)
)
"""

run_update(create_table, db2, 'create')

run_query('DESCRIBE members', db2)

,Field,Type,Null,Key,Default,Extra
0,member_id,b'int',NO,PRI,None,auto_increment
1,fname,b'varchar(50)',NO,,None,
2,lname,b'varchar(50)',NO,,None,
3,instrument,b'varchar(50)',NO,,None,


### ALTER a table to add a new field (column)
I forgot to add the band name field, so I use the ALTER keyword to do so

In [48]:
# Add the field band_name to the members table

create_table = f"""
ALTER TABLE members
ADD COLUMN band_name VARCHAR(75)
"""
run_update(create_table, db2, 'alter')
run_query('DESCRIBE members', db2)

,Field,Type,Null,Key,Default,Extra
0,member_id,b'int',NO,PRI,None,auto_increment
1,fname,b'varchar(50)',NO,,None,
2,lname,b'varchar(50)',NO,,None,
3,instrument,b'varchar(50)',NO,,None,
4,band_name,b'varchar(75)',YES,,None,


### MODIFY altered table 
I forgot to make the band_names field NOT NULL so below I modify the column to rectify the situation, note that COLUMN is not needed after MODIFY or ADD from above, but it makes the query more understandable

In [49]:
# Add the field band_name to the members table

create_table = f"""
ALTER TABLE members
MODIFY COLUMN band_name VARCHAR(75) NOT NULL
"""
run_update(create_table, db2, 'alter')
run_query('DESCRIBE members', db2)

,Field,Type,Null,Key,Default,Extra
0,member_id,b'int',NO,PRI,None,auto_increment
1,fname,b'varchar(50)',NO,,None,
2,lname,b'varchar(50)',NO,,None,
3,instrument,b'varchar(50)',NO,,None,
4,band_name,b'varchar(75)',NO,,None,


### INSERT records into a table

In [50]:
# Here I insert 4 band members into the table

insert_data = f"""
INSERT INTO members (fname, lname, instrument, band_name)
VALUES
  ('Paul', 'Bullard', 'Guitar', 'Roof Diver'),
  ('Cameron', 'McClain', 'Bass', 'Roof Diver'),
  ('Eric', 'Elliot', 'Guitar', 'Roof Diver'),
  ('Rodney', 'Walton', 'Drums', 'Roof Diver');
"""

query = """
SELECT * FROM members;
"""
run_update(insert_data, db2, 'insert')

run_query(query, db2)

,member_id,fname,lname,instrument,band_name
0,1,Paul,Bullard,Guitar,Roof Diver
1,2,Cameron,McClain,Bass,Roof Diver
2,3,Eric,Elliot,Guitar,Roof Diver
3,4,Rodney,Walton,Drums,Roof Diver


### UPDATING Table Values
The UPDATE keyword along with SET is used to update existing values in a table
In the example below I create a new column for vocals and make the default = 'no'.
Then I update this to yes for memeber who sing

In [51]:
# Create a new column for vocals
add_column = f"""
ALTER TABLE members
ADD COLUMN vocals VARCHAR(5) DEFAULT 'no'
"""

run_update(add_column, db2, 'alter')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,no
1,2,Cameron,McClain,Bass,Roof Diver,no
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no


In [52]:
# Update the vocals column to 'yes' for singing members
update_vox = f"""
UPDATE members
SET vocals = 'yes'
WHERE member_id = 1 OR member_id = 2;
"""

run_update(update_vox, db2, 'update')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,yes
1,2,Cameron,McClain,Bass,Roof Diver,yes
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no


### DELETE a record
This is as simple as it sounds, but I add another few rows to the dataset beforehand

In [53]:
# Adding in a few more rows (most with incorrect data for later examples)
insert_data = f"""
INSERT INTO members (fname, lname, instrument, band_name, vocals)
VALUES
  ('John', 'Lennon', 'Guitar', 'Rolling Stones', 'yes'),
  ('Paul', 'McCartney', 'Drums', 'The Beatles', 'no'),
  ('Ringo', 'Star', 'Drums', 'The Beatles', 'yes'),
  ('George', 'Hamilton', 'Guitar', 'The Beatles', 'yes'),
  ('Jack', 'Spratt', 'Triangle', 'Yo Yo Yo', 'no');
"""

query = """
SELECT * FROM members;
"""
run_update(insert_data, db2, 'insert')

run_query(query, db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,yes
1,2,Cameron,McClain,Bass,Roof Diver,yes
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no
4,5,John,Lennon,Guitar,Rolling Stones,yes
5,6,Paul,McCartney,Drums,The Beatles,no
6,7,Ringo,Star,Drums,The Beatles,yes
7,8,George,Hamilton,Guitar,The Beatles,yes
8,9,Jack,Spratt,Triangle,Yo Yo Yo,no


In [54]:
# Delete the last row because it it's not valid 
update_vox = f"""
DELETE FROM members
WHERE member_id = 9;
"""

run_update(update_vox, db2, 'delete')
run_query('SELECT * FROM members', db2)

,member_id,fname,lname,instrument,band_name,vocals
0,1,Paul,Bullard,Guitar,Roof Diver,yes
1,2,Cameron,McClain,Bass,Roof Diver,yes
2,3,Eric,Elliot,Guitar,Roof Diver,no
3,4,Rodney,Walton,Drums,Roof Diver,no
4,5,John,Lennon,Guitar,Rolling Stones,yes
5,6,Paul,McCartney,Drums,The Beatles,no
6,7,Ringo,Star,Drums,The Beatles,yes
7,8,George,Hamilton,Guitar,The Beatles,yes


---
<a id="s3"></a>
## 3. Combining Tables
There two main ways to combine tables in SQL. JOIN statements combine tables by columns and tables are joined by rows using UNION. Both are discussed in detail below:

### Joins
Joins are used to combine tables (column-wise) based off of a common variables (usually primary and/or foreign keys). There are 4 main types of Joins:

* **Inner Join** - Combines tables where only common variable matches
* **Left Join** - Combines everything from left table with matching common variable from right
* **Right Join** - Combines everything from right table with matching common variable from 
* **Self Join** - An inner join that joins values from the same table (usually the second table is cretead as an alias), an example of this would be having a table with an employee id and a boss id, then combining the employee with the boss
* **Full Outer Join** - Combines everything from both tables regardless of match

Left/Right/and Outer joins can also be adjusted slightly for different results froma above. See the chart below for a visual on all types of joins:

<img src='data/images/sql1.png' width='700' >

## Setting up a database for the join examples
For the join examples, I am using a sample database representing a business that sells scaled models of different kinds of vehicles including cars, buses, planes, motorcycles, etc., below is the table relationships:

NOTE: The business has Employees who each have their own Customers 

<img src='data/images/sql2.png' width='700' >

In [33]:
run_query('select * from employees limit 5', db3)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002.0,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056.0,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056.0,Sale Manager (EMEA)


### Inner Join
Combines only those valuse that match between both tables

In [34]:
# In this example I use an inner join to match the employee with their respective customers

query = """
SELECT
e.employeeNumber, 
CONCAT(e.firstName ," ", e.lastName) AS 'employee',
e.jobTitle,
c.customerName,
c.country AS 'customer_location'
FROM employees AS e

INNER JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber

ORDER BY employeeNumber ASC;
"""

run_query(query, db3)

,employeeNumber,employee,jobTitle,customerName,customer_location
0,1165,Leslie Jennings,Sales Rep,Signal Collectibles Ltd.,USA
1,1165,Leslie Jennings,Sales Rep,The Sharp Gifts Warehouse,USA
2,1165,Leslie Jennings,Sales Rep,Mini Gifts Distributors Ltd.,USA
3,1165,Leslie Jennings,Sales Rep,Technics Stores Inc.,USA
4,1165,Leslie Jennings,Sales Rep,Mini Wheels Co.,USA
...,...,...,...,...,...
95,1702,Martin Gerard,Sales Rep,"Corrida Auto Replicas, Ltd",Spain
96,1702,Martin Gerard,Sales Rep,"Vida Sport, Ltd",Switzerland
97,1702,Martin Gerard,Sales Rep,Enaco Distributors,Spain
98,1702,Martin Gerard,Sales Rep,"Iberia Gift Imports, Corp.",Spain


### Left Join
Left joins (and left outer joins) keep all values from the left table (i.e. the one prior to the join statement) regardless of id match. This is used when you need all the records in the first table but only those that match from the second.

In [38]:
# Below I perform the same query as above, however this time the executives
# are included even though they don't have customers because they are in 
# the left table

query = """
SELECT
e.employeeNumber, 
CONCAT(e.firstName ," ", e.lastName) AS 'employee',
e.jobTitle,
c.customerName,
c.country AS 'customer_location'
FROM employees AS e

LEFT JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber

ORDER BY employeeNumber ASC;
"""

run_query(query, db3)

,employeeNumber,employee,jobTitle,customerName,customer_location
0,1002,Diane Murphy,President,None,None
1,1056,Mary Patterson,VP Sales,None,None
2,1076,Jeff Firrelli,VP Marketing,None,None
3,1088,William Patterson,Sales Manager (APAC),None,None
4,1102,Gerard Bondur,Sale Manager (EMEA),None,None
...,...,...,...,...,...
103,1702,Martin Gerard,Sales Rep,"Vida Sport, Ltd",Switzerland
104,1702,Martin Gerard,Sales Rep,CAF Imports,Spain
105,1702,Martin Gerard,Sales Rep,Precious Collectables,Switzerland
106,1702,Martin Gerard,Sales Rep,"Corrida Auto Replicas, Ltd",Spain


### Right Join
Same functionality as left join, but in reverse (i.e., keeps everything in table AFTER the join statment and only the matches from the first)

In [39]:
# Note with the right join, there are cutomers and locations that do no match to an employee
# so when that happens NaN or None is returned. 

query = """
SELECT
e.employeeNumber, 
CONCAT(e.firstName ," ", e.lastName) AS 'employee',
e.jobTitle,
c.customerName,
c.country AS 'customer_location'
FROM employees AS e

RIGHT JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber

ORDER BY employeeNumber ASC;
"""

run_query(query, db3)

,employeeNumber,employee,jobTitle,customerName,customer_location
0,NaN,None,None,Havel & Zbyszek Co,Poland
1,NaN,None,None,Porto Imports Co.,Portugal
2,NaN,None,None,"Asian Shopping Network, Co",Singapore
3,NaN,None,None,Natürlich Autos,Germany
4,NaN,None,None,ANG Resellers,Spain
...,...,...,...,...,...
117,1702.0,Martin Gerard,Sales Rep,"Vida Sport, Ltd",Switzerland
118,1702.0,Martin Gerard,Sales Rep,CAF Imports,Spain
119,1702.0,Martin Gerard,Sales Rep,Precious Collectables,Switzerland
120,1702.0,Martin Gerard,Sales Rep,"Corrida Auto Replicas, Ltd",Spain


### Cross-Join
A cross join is created by using CROSS JOIN when joining two tables OR by ommiting the ON keyword when using INNER JOIN to join two tables. 

Cross joins create a Cartesian product, which is every possible combination of 1 row from the first table and 1 row from the second (for example: the cross join for a table with three rows ('a', 'b', and 'c') and a table with four rows (say 1, 2, 3, 4) would have 12 rows)

In [9]:
# Step 1: Join the 'productLines' and 'products' tables
#         by matching product names with the product line
#         notice that even though productLine = 'Planes'
#         other product lines are being returned. This is
#         because the keyword ON is not used

query = """
SELECT p.productName, p.productLine
FROM productLines as pl CROSS JOIN products AS p
WHERE pl.productLine = 'Planes'
LIMIT 5;
"""
run_query(query, db3)

,productName,productLine
0,1969 Harley Davidson Ultimate Chopper,Motorcycles
1,1952 Alpine Renault 1300,Classic Cars
2,1996 Moto Guzzi 1100i,Motorcycles
3,2003 Harley-Davidson Eagle Drag Bike,Motorcycles
4,1972 Alfa Romeo GTA,Classic Cars


### Inner Join Example 2

In [65]:
# Step 2 : Note here the ON clause is added which is 
#          needed to actually only match Planes

query = """
SELECT p.productName, p.productLine
FROM productLines as pl INNER JOIN products AS p
ON pl.productLine = p.productLine
WHERE pl.productLine = 'Planes'
LIMIT 5;
"""
run_query(query, db3)

,productName,productLine
0,1980s Black Hawk Helicopter,Planes
1,P-51-D Mustang,Planes
2,1928 British Royal Navy Airplane,Planes
3,1900s Vintage Bi-Plane,Planes
4,Corsair F4U ( Bird Cage),Planes


### Multiple-Joins
In many cases you will need to join multiple tables rather than just two. If multiple tables aren't joine properly, then erroneous data will result that can be large amounts of unnecessary data being returned. 

The below example joins product lines all the way to employees in the dataset, see the link below for details regarding this section:

[Tutorial Video For This Dataset](https://www.youtube.com/watch?v=CaRaYQiCCE0&list=PLyuRouwmQCjlXvBkTfGeDTq79r9_GoMt9&index=30&ab_channel=SteveGriffith-Prof3ssorSt3v3)

In [87]:
# In this example looking to get where:
# productLine = 'Planes' & (employee location) city = 'Sydney'

# In order to get from 'productLine' to 'city', 7 tables must be traversed (see diagram)
# in this order:
# productLines --> products --> orderdetails --> orders --> customers --> employees --> offices

query = """
SELECT p.productName, p.productLine, c.customerName, off.city AS emplyee_loc

-- first join (gets all plane products)
FROM productLines as pl INNER JOIN products AS p
ON pl.productLine = p.productLine

-- second join (connects prior results with orderdetails)
INNER JOIN orderdetails AS od
USING(productCode)

-- third join (connect prior results with orders)
INNER JOIN orders AS o
USING(orderNumber)

-- fourth join (connects prior results to customers)
INNER JOIN customers AS c
USING(customerNumber)

-- fifth join (connect prior results to employees)
INNER JOIN employees AS e
ON e.employeeNumber = c.salesRepEmployeeNumber

-- sixth join
INNER JOIN offices AS off
USING(officeCode)

WHERE pl.productLine = 'Planes' AND off.city = 'Sydney'
limit 5;
"""
run_query(query, db3)

,productName,productLine,customerName,emplyee_loc
0,P-51-D Mustang,Planes,"Australian Collectors, Co.",Sydney
1,1928 British Royal Navy Airplane,Planes,"Australian Collectors, Co.",Sydney
2,1900s Vintage Tri-Plane,Planes,"Australian Collectors, Co.",Sydney
3,American Airlines: B767-300,Planes,"Australian Collectors, Co.",Sydney
4,America West Airlines B757-200,Planes,"Australian Collectors, Co.",Sydney


## Perfect Join Example
This example matches customers with their orders
The table includes the customerName, orderDate, status (of order), productName, and productLine

In order to get these results, 4 tables have to be joined together.

The end result is a perfect example of the kind of table that could then be used with a 
Tableau dashboard for visual analysis and filtering

In [28]:
# match customers with their orders based on productLine

# table join order: customer --> orders --> orderdetails --> 

query = """
SELECT c.customerName, c.country, o.orderDate, o.status, p.productName, p.productLine
FROM customers as c 
LEFT JOIN orders AS o USING (customerNumber) 
INNER JOIN orderdetails AS od USING (orderNumber)
INNER JOIN products AS p USING(productCode)
ORDER BY 'customerName' desc
"""
run_query(query, db3)

,customerName,country,orderDate,status,productName,productLine
0,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1917 Grand Touring Sedan,Vintage Cars
1,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1911 Ford Town Car,Vintage Cars
2,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars
3,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1936 Mercedes Benz 500k Roadster,Vintage Cars
4,"Blauer See Auto, Co.",Germany,2003-01-09,Shipped,1932 Model A Ford J-Coupe,Vintage Cars
...,...,...,...,...,...,...
2991,La Rochelle Gifts,France,2005-05-31,In Process,1962 Volkswagen Microbus,Trucks and Buses
2992,La Rochelle Gifts,France,2005-05-31,In Process,1958 Chevy Corvette Limited Edition,Classic Cars
2993,La Rochelle Gifts,France,2005-05-31,In Process,1980’s GM Manhattan Express,Trucks and Buses
2994,La Rochelle Gifts,France,2005-05-31,In Process,1954 Greyhound Scenicruiser,Trucks and Buses


### Union

UNION statements are used to create a combined tables (row-wise) using multiple select statements.

There are a few imporant caveats including:
* Every select statement within UNION must have the same # of columns
* The columns must be of the same data type
* The columns in each select statement must also be in order

There are 3 types of UNION statements:
* **UNION** - selects only disctinct values by default
* **UNION DISTINCT** - same as UNION, just explicitly declared
* **UNION ALL** - allows duplicate values

In [14]:
# This query combines employees and customer point of contact individuals

query = """
SELECT employeeNumber as id, lastName as l_name, firstName as f_name
FROM employees

Where firstName = 'Leslie'

UNION

SELECT customerNumber as id, contactLastName as l_name, contactFirstName  as f_name
from customers

where contactFirstName = 'Sue'

"""
run_query(query, db3)


,id,l_name,f_name
0,1165,Jennings,Leslie
1,1166,Thompson,Leslie
2,450,Frick,Sue
3,487,Taylor,Sue


---
<a id="s4"></a>
## 4. FUNCTIONS
There are a number of built-in function types in SQL. 

### Aggregate Functions
Functions that perform some form of calculation over entire sections of data (i.e. entire columns ect.), some exampels are AVG(), MIN(), MAX(), COUNT()

Below are a few examples of aggregate functions

In [68]:
# Get the Average runtime for a movie
query = """
SELECT AVG(runtime)
FROM movies
"""
run_query(query, db1)

,AVG(runtime)
0,100.119


In [69]:
# Get the movie with the longest runtime
query = """
SELECT title, MAX(runtime)
FROM movies
"""
run_query(query, db1)

,title,MAX(runtime)
0,Norm of the North: King Sized Adventure,228


In [70]:
# COUNT the number of movies with runtime > the average runtime of 100.119 minutes
# Get the movie with the longest runtime
query = """
SELECT COUNT(*)
FROM movies
WHERE runtime > 100
"""
run_query(query, db1)

,COUNT(*)
0,1863


### GROUP BY
Used in conjunction with aggregate functions performed on multiple columns

In [75]:
# The query below is incorrect because an aggregate function is being
# performed on multiple columns with no identifier for which
# column to perform the aggregation on

# This query just adds all the movies up and assing them to
# the first director 
query = """
SELECT COUNT(id) as total, director
FROM movies
"""
run_query(query, db1)

,total,director
0,4125,Richard Finn


In [76]:
# The keyword GROUP BY tells SQL how to aggregate the 
# desired data, in this case, count the number of movies per director
query = """
SELECT COUNT(id) as count, director
FROM movies
GROUP BY director
limit 10
"""
run_query(query, db1)

,count,director
0,1,Richard Finn
1,1,Fernando Lebrija
2,1,Gabe Ibanez
3,1,Rodrigo Toro
4,1,Henrik Ruben Genz
5,1,Jose Miguel Contreras
6,2,Daniel Alfredson
7,3,Munjal Shroff
8,3,Tilak Shetty
9,1,Gaspar Noe


In [100]:
# This example uses the model business data to count the number
# of customers per employee

# Step 1: Create Query To Connect The Two Tables Appropriately
#         Below I match employees with customer by employeeNumber

query = """
SELECT
e.employeeNumber, 
CONCAT(e.firstName ," ", e.lastName) AS 'e.employee',
c.customerName
FROM employees as e

INNER JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber
limit 5;
"""

run_query(query, db3)

,employeeNumber,e.employee,customerName
0,1370,Gerard Hernandez,Atelier graphique
1,1166,Leslie Thompson,Signal Gift Stores
2,1611,Andy Fixter,"Australian Collectors, Co."
3,1370,Gerard Hernandez,La Rochelle Gifts
4,1504,Barry Jones,Baane Mini Imports


In [132]:
# Step 2: Add the aggregate function and group by accordingly
#         In this case I want to count the customers and
#         group by employee (number of customers per employee)

query = """
SELECT 
CONCAT(e.firstName ," ", e.lastName) AS 'e.employee',
COUNT(c.customerName) as num_customers

FROM employees as e

INNER JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber

GROUP BY CONCAT(e.firstName ," ", e.lastName)
"""

run_query(query, db3)

,e.employee,num_customers
0,Gerard Hernandez,7
1,Leslie Thompson,6
2,Andy Fixter,5
3,Barry Jones,9
4,Leslie Jennings,6
5,George Vanauf,8
6,Pamela Castillo,10
7,Loui Bondur,6
8,Mami Nishi,5
9,Foon Yue Tseng,7


In [157]:
# This example sums the number of customers (I did this as a double check)
# It is using a with/as clause which is a form of sub-query
query = """

WITH customer_count AS (

SELECT 
COUNT(c.customerName) as num_customers

FROM employees as e

INNER JOIN customers AS c
ON e.employeeNumber = c.salesRepEmployeeNumber

GROUP BY CONCAT(e.firstName ," ", e.lastName)
)

SELECT SUM(num_customers) FROM customer_count

"""

run_query(query, db3)

,SUM(num_customers)
0,100.0


# TODO 
#### Create an order count column (i.e. each order is 1 out of some #), to do this, orderDate may need to be counted

In [55]:
# ADD A CALCULATED COLUMN AT THE END WITH THE SUM PRODCUT COUNT FOR EACH ORDER DATE
# In the below query, Online Diecast Creations ordered 4 products on 2003-01-06 so each row is 1 of 4
query = """
SELECT c.customerName, c.country, o.orderDate, o.status, p.productName, p.productLine
FROM customers as c 
LEFT JOIN orders AS o USING (customerNumber) 
INNER JOIN orderdetails AS od USING (orderNumber)
INNER JOIN products AS p USING(productCode)
ORDER BY 'customerName' desc
"""
run_query(query, db3)

,customerName,country,orderDate,status,productName,productLine
0,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1917 Grand Touring Sedan,Vintage Cars
1,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1911 Ford Town Car,Vintage Cars
2,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars
3,Online Diecast Creations Co.,USA,2003-01-06,Shipped,1936 Mercedes Benz 500k Roadster,Vintage Cars
4,"Blauer See Auto, Co.",Germany,2003-01-09,Shipped,1932 Model A Ford J-Coupe,Vintage Cars
...,...,...,...,...,...,...
2991,La Rochelle Gifts,France,2005-05-31,In Process,1962 Volkswagen Microbus,Trucks and Buses
2992,La Rochelle Gifts,France,2005-05-31,In Process,1958 Chevy Corvette Limited Edition,Classic Cars
2993,La Rochelle Gifts,France,2005-05-31,In Process,1980’s GM Manhattan Express,Trucks and Buses
2994,La Rochelle Gifts,France,2005-05-31,In Process,1954 Greyhound Scenicruiser,Trucks and Buses


In [43]:
query = """
SELECT c.customerName, c.country, o.orderDate, o.status
FROM customers as c 
LEFT JOIN orders AS o USING (customerNumber) 

ORDER BY 'customerName' desc
"""
run_query(query, db3)

,customerName,country,orderDate,status
0,Atelier graphique,France,2003-05-20,Shipped
1,Atelier graphique,France,2004-09-27,Shipped
2,Atelier graphique,France,2004-11-25,Shipped
3,Signal Gift Stores,USA,2003-05-21,Shipped
4,Signal Gift Stores,USA,2004-08-06,Shipped
...,...,...,...,...
345,Diecast Collectables,USA,2004-04-26,Shipped
346,Kelly's Gift Shop,New Zealand,2003-07-07,Shipped
347,Kelly's Gift Shop,New Zealand,2003-11-11,Cancelled
348,Kelly's Gift Shop,New Zealand,2004-12-16,Shipped


### Search for easier ways of doing the above summation

### Function Variables


In [25]:
run_query('select * from movies limit 2', db1)

,id,title,director,release_date,runtime,rating,category
0,81145628,Norm of the North: King Sized Adventure,Richard Finn,2019,90,TV-PG,Children & Family Movies
1,80125979,#realityhigh,Fernando Lebrija,2017,99,TV-14,Comedies


---
<a id="s5"></a>
## 5. SubQueries (IN)
A Subquery (aka: Inner query, aka: Nested query) is a query within another SQL query and embedded within the WHERE clause. Subqueries are used to return data that will be used in the main query as a condition to further restrict the data to be retrieved.

Subqueries can be used with the SELECT, INSERT, UPDATE, and DELETE statements along with the operators like =, <, >, >=, <=, IN, BETWEEN, etc.

There are a few caveats to subqueries including:

* Subqueries must be enclosed within parentheses
* A subquery can have only one column in the SELECT clause, unless multiple columns are in the main query for the subquery to compare its selected columns
* An ORDER BY command cannot be used in a subquery, although the main query can use an ORDER BY. The GROUP BY command can be used to perform the same function as the ORDER BY in a subquery
* Subqueries that return more than one row can only be used with multiple value operators such as the IN operator.
* The SELECT list cannot include any references to values that evaluate to a BLOB, ARRAY, CLOB, or NCLOB
* A subquery cannot be immediately enclosed in a set function
* The BETWEEN operator cannot be used with a subquery. However, the BETWEEN operator can be used within the subquery

In [103]:
# Normal method for requesting multiple column values
query1 = f"""
SELECT *
FROM `movies`
WHERE (release_date = 2019 OR release_date = 2020)
ORDER BY release_date DESC
LIMIT {limit}
"""

# SubQuery method
query2 = f"""
SELECT *
FROM `movies`
WHERE release_date IN (2017, 2020)
ORDER BY release_date DESC
LIMIT {limit}
"""

run_query(query2)

,id,title,director,release_date,runtime,rating,category
0,81006825,All the Freckles in the World,Yibran Asuad,2020,90,TV-14,Comedies
1,81127902,A Fall from Grace,Tyler Perry,2020,121,TV-MA,Dramas
2,80233408,"Live Twice, Love Once",Maria Ripoll,2020,102,TV-MA,Comedies
3,81060049,Leslie Jones: Time Machine,David Benioff,2020,66,TV-MA,Stand-Up Comedy
4,81214114,Bulletproof 2,Don Michael Paul,2020,97,TV-MA,Action & Adventure


### Subqueries Used as Filters
Subqueries can be used to filter one table from values of another (NOTE BOTH TABLES MUST MATCH ON SAID VALUE)

In [175]:
# In the example below, the movies table is region filtered by id

query1 = """
DESCRIBE region
"""
print('The Region Table Has 2 Fields')
print(run_query(query1))


query2 = f"""
SELECT DISTINCT region_code
FROM `region`
ORDER BY region_code ASC
"""
print()
print()
print('There are 10 unique region_code values:')
print(run_query(query2).region_code.unique())


query3 = f"""
SELECT * FROM movies
WHERE id IN (SELECT id FROM region WHERE region_code = 4)
AND rating = 'R'
ORDER BY runtime ASC
"""
run_query(query3)

The Region Table Has 2 Fields
         Field    Type Null Key Default Extra
0           id  b'int'  YES        None      
1  region_code  b'int'  YES        None      


There are 10 unique region_code values:
[ 1  2  3  4  5  6  7  8  9 10]


,id,title,director,release_date,runtime,rating,category
0,60024976,Jerry Seinfeld: Comedian,Christian Charles,2002,81,R,Documentaries
1,70211216,Kevin Hart: Laugh at My Pain,Leslie Small,2011,89,R,Stand-Up Comedy
2,80037280,The Witch,Robert Eggers,2015,92,R,Horror Movies
3,80017286,While We're Young,Noah Baumbach,2015,97,R,Comedies
4,80240972,The Kindergarten Teacher,Sara Colangelo,2018,97,R,Dramas
5,80028357,"Love, Rosie",Christian Ditter,2014,103,R,Comedies
6,70024088,Brick,Rian Johnson,2005,110,R,Independent Movies
7,485218,Executive Decision,Stuart Baird,1996,133,R,Action & Adventure
8,80175694,Mudbound,Dee Rees,2017,135,R,Dramas


### SubQuries Within SubQueries
Subqueries can be chained, but this can get complicated quickly and is not recommended

# DIAL WITH/AS

---
<a id="s6"></a>
## 6. User Stored Funtions, Variables, Scripts, and Stored Procedures

### Stored FUNTION
Functions are created in SQL in a similar fashion to other programing languages, however, the big difference is that all paramaeter and return values must have their types explicitly declared.

When functions are created they are stored in their own section within the database separate from the tables.

In [8]:
# Here a function is created to concatenate a movie title with it's director
# Note that the function parameters (movie_title & dir_name) both have type declared
# and also the returnef value type has to be declared (here CHAR(100))
# DETERMINISTIC - functions always return the same result any time they are called
# with a specific set of input values and given the same state of the database

# Creating the function (will be stored like a table in the db)
query = f"""
CREATE FUNCTION rating_director(rating) CHAR(50), dir_name CHAR(50))
RETURNS CHAR(100) DETERMINISTIC
RETURN CONCAT(rating, ';',dir_name);
"""
#run_update(query, db1,'insert')
print('function added to db')


# Using the function in a query
# Note rating_director is concatenated rating and director
query2 = f"""
Select
title, 
rating_director(rating, director),
release_date
From movies
limit 5;
"""
run_query(query2, db1)

function added to db


,title,"rating_director(rating, director)",release_date
0,Norm of the North: King Sized Adventure,TV-PG;Richard Finn,2019
1,#realityhigh,TV-14;Fernando Lebrija,2017
2,Automata,R;Gabe Ibanez,2014
3,Fabrizio Copano: Solo pienso en mi,TV-MA;Rodrigo Toro,2017
4,Good People,R;Henrik Ruben Genz,2014


### CREATING USER VARIABLES
user variables can be created with the SET keyword (note: SET also used to make changes to table data)

In [10]:
# Variables created using: SET @var_name = desired value
# This doesn't work in python because the cursor can
# only run 1 query at a time, here there are two going on
# one for SET when the variable is declared and the Select after

# This does work in MySQL workbench

query = """
SET @date = 2014;
SELECT title, director, release_date
FROM movies
WHERE release_date = @date;
"""
run_query(query, db1)

Error: could not connect to netflix database.


### SCRIPTS



### STORED PROCEDURES
Stored functions allow for a single query run, however stored procedures allow for multiple queries to be run at once with one function call

#### NOTE: The below code works in MySQL workbench but not here. It has something to do with the cursor and the execution
## DIAL THAT AT SOME POINT

In [49]:
# This creates the procedure
procedure = """

DELIMITER //

CREATE PROCEDURE band.band_location (IN name VARCHAR(50)) 
BEGIN

SELECT DISTINCT m.band_name, l.location
from members as m
inner join locations as l
using (location_id)
where m.band_name = name;

END //
DELIMITER ;
"""


call_proc = """
CALL band_location('Roof Diver');
"""

# this runs the procedure
run_query(call_proc, db2);

Error: could not connect to band database.


## Triggers
In SQL there are triggers used for events similar to event handlers in javascript. 

NONE OF THE BELOW QUERIES WORK (Even in MySQL), it could be several things, most likely the db I am using is corrupted on some level. For whatever reason when I try to run the queries the server cannot be connected to.

NOTE THAT THE SYNTAX IS CORRECT HOWEVER

In [ ]:
query="""

- Drops Triggers if they exists already 
DROP TRIGGER IF EXISTS insert_band_location;
DROP TRIGGER IF EXISTS update_band_location;


-- This trigger will convert any newly inserted band location to uppercase

CREATE TRIGGER insert_band_location
BEFORE INSERT ON locations
FOR EACH ROW
SET NEW.location = UPPER(NEW.location);


-- This trigger will c
CREATE TRIGGER update_band_location
BEFORE UPDATE ON locations
FOR EACH ROW
SET NEW.location = LOWER(NEW.location);
"""
